In [ ]:
%reload_ext autoreload
%autoreload 2

from util.scenario import Scenario
import numpy as np
import matplotlib.pyplot as plt
from pyomo_models.base import BaseModel
from pyomo_models.multi_uavs import MultiUavModel
from pyomo.opt import SolverFactory
from simulate.simulate import Simulation, WhiteNoiseEnvironment
from tqdm.notebook import tqdm
import pandas as pd

In [ ]:
sc = Scenario.from_file("scenarios/single_longer_path.yaml")

# PARAMETERS
B_min = 0.2
B_max = 1
B_start = [1]
v = [1]
r_charge = np.array([1])
r_deplete = np.array([0.35])

parameters = {p: eval(p) for p in ['B_min', 'B_max', 'B_start', 'r_charge', 'r_deplete', 'v']}
model = BaseModel(sc, parameters)

solution = SolverFactory('gurobi').solve(model) 
time_taken = solution['solver'][0]['time']
print(f"Took {time_taken:.3f}s to find charging schedule")

In [ ]:
env = WhiteNoiseEnvironment()

### Default strategy

In [ ]:
nsims = 10000
np.random.seed(0)

total_charges = []
successes = []

sim = Simulation.from_base_model(model, 0, env=env)

for i in tqdm(range(nsims)):
    charges, timestamps, path, path_arrivals, charging_windows, ok = sim.simulate()
    successes.append(ok)
    total_charges += [charges]
successes = pd.Series(successes)

In [ ]:
_, ax = plt.subplots(figsize=(8,1.5), dpi=150)
ok = sim.plot_charge(ax=ax)
plt.title("Finished" if ok else "Not finished")
plt.show()

In [ ]:
successes.sum()